In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

seed = 42

In [2]:
import nbimporter

import pre_processing
import feature_selection

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from feature_selection.ipynb


In [44]:
def escribir_respuesta(ids,predicciones):
    with open("respuesta.csv",'w') as archivo:
        archivo.write("id,target\n")
        for i in range(len(ids)):
            linea = f"{int(ids[i])},{predicciones[i]}"
            archivo.write(f"{linea}\n")

In [3]:
train,test = pre_processing.load_featured_datasets()

In [4]:
train = feature_selection.get_selected_dataframe(train)
test = feature_selection.get_selected_dataframe(test, precio=False)

# Problema de clasificacion: queremos ver si es outlier o no
### Primer modelo

In [6]:
mean = train['precio'].describe()[1]
std = train['precio'].describe()[2]

In [7]:
def bin_std(x, sup, inf):
    if ((x<sup) & (x>inf)):
        return 1
    return 0

In [8]:
train['target'] = train['precio'].map(lambda x: bin_std(x, mean+std, mean-std))

In [10]:
train['target'].value_counts()

1    201030
0     38970
Name: target, dtype: int64

In [11]:
# Entrenamos el modelo

In [13]:
train_2 = train.drop('precio', axis=1)

In [16]:
X = train_2.drop('target', axis=1)
Y = train_2['target']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [18]:
import lightgbm as lgb

In [17]:
lightgbm = {'objective': 'binary',
            'num_leaves': 36,
            'metric':'auc',
            'n_estimators': 70,
            'min_split_gain': 0.01,
            'min_child_weight': 5.00001,
            'max_depth': 4,
            'learning_rate': 0.05,
            'lambda_l2': 0,
            'feature_fraction': 0.7000000000000001,
            'bagging_fraction': 1.0}

In [20]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(lightgbm, d_train, valid_sets=watchlist, verbose_eval=1)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.85031
[2]	valid_0's auc: 0.871628
[3]	valid_0's auc: 0.872996
[4]	valid_0's auc: 0.875698
[5]	valid_0's auc: 0.892512
[6]	valid_0's auc: 0.894525
[7]	valid_0's auc: 0.895374
[8]	valid_0's auc: 0.900426
[9]	valid_0's auc: 0.900607
[10]	valid_0's auc: 0.902773
[11]	valid_0's auc: 0.904824
[12]	valid_0's auc: 0.907756
[13]	valid_0's auc: 0.908252
[14]	valid_0's auc: 0.910331
[15]	valid_0's auc: 0.912091
[16]	valid_0's auc: 0.912711
[17]	valid_0's auc: 0.913537
[18]	valid_0's auc: 0.915042
[19]	valid_0's auc: 0.915538
[20]	valid_0's auc: 0.916125
[21]	valid_0's auc: 0.916718
[22]	valid_0's auc: 0.917486
[23]	valid_0's auc: 0.91789
[24]	valid_0's auc: 0.918444
[25]	valid_0's auc: 0.919666
[26]	valid_0's auc: 0.920561
[27]	valid_0's auc: 0.920683
[28]	valid_0's auc: 0.921344
[29]	valid_0's auc: 0.92173
[30]	valid_0's auc: 0.922091
[31]	valid_0's auc: 0.922329
[32]	valid_0's auc: 0.923173
[33]	valid_0's auc: 0.923642
[34]	valid_0's auc: 0.924422
[35]	valid_0's auc: 0.9250

In [23]:
test_predicted = reg.predict(test)

In [24]:
f = np.vectorize(lambda x: 1 if (x>0.5) else 0)
test_predicted = f(test_predicted)

In [25]:
test['target'] = test_predicted

In [27]:
test['target'].value_counts()

1    53287
0     6713
Name: target, dtype: int64

In [29]:
train['target'].value_counts()

1    201030
0     38970
Name: target, dtype: int64

In [30]:
# Ahora lo que queremos es separar en dos el problema: MODELO para los datos confiables, y otro modelo
# para los datos no confiables, y luego juntar todo.

In [31]:
train_a = train.loc[train['target'] == 1].drop('target', axis=1).copy()
train_b = train.loc[train['target'] == 0].drop('target', axis=1).copy()

In [37]:
test_a = test.loc[test['target'] == 1].drop('target', axis=1).copy()
test_b = test.loc[test['target'] == 0].drop('target', axis=1).copy()

# Predicciones para datos confiables:

In [34]:
# Entrenamos el modelo:
X = train_a.drop('precio', axis=1)
Y = train_a['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [35]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14,
    'num_leaves': 120,
    #'learning_rate': 0.02,
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 100}
n_estimators=20000

In [36]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, valid_sets=watchlist, verbose_eval=50)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds
[50]	valid_0's l1: 382895
[100]	valid_0's l1: 343759
[150]	valid_0's l1: 334314
[200]	valid_0's l1: 329693
[250]	valid_0's l1: 326999
[300]	valid_0's l1: 325185
[350]	valid_0's l1: 323419
[400]	valid_0's l1: 322047
[450]	valid_0's l1: 320659
[500]	valid_0's l1: 319511
[550]	valid_0's l1: 318476
[600]	valid_0's l1: 317524
[650]	valid_0's l1: 316727
[700]	valid_0's l1: 316114
[750]	valid_0's l1: 315391
[800]	valid_0's l1: 314796
[850]	valid_0's l1: 314198
[900]	valid_0's l1: 313575
[950]	valid_0's l1: 313047
[1000]	valid_0's l1: 312657
[1050]	valid_0's l1: 312265
[1100]	valid_0's l1: 311848
[1150]	valid_0's l1: 311426
[1200]	valid_0's l1: 310927
[1250]	valid_0's l1: 310559
[1300]	valid_0's l1: 310333
[1350]	valid_0's l1: 310001
[1400]	valid_0's l1: 309772
[1450]	valid_0's l1: 309473
[1500]	valid_0's l1: 309264
[1550]	valid_0's l1: 309067
[1600]	valid_0's l1: 308787
[1650]	valid_0's l1: 308504
[1700]	valid_0's l1: 308273
[1750]

In [41]:
# Realizamos las predicciones para test_a
pred_test_a = reg.predict(test_a.values)

In [42]:
pred_test_a

array([1068873.65659347, 2225691.17965524, 1085875.8559637 , ...,
        764217.32817345, 1491791.99554189, 2417493.41916187])

In [45]:
test_a['precio'] = pred_test_a

In [46]:
test_a = test_a.reset_index()[['id', 'precio']]

In [49]:
# tenemos test a
test_a.head()

,id,precio
0,51775,1.068874e+06
1,115253,2.225691e+06
2,299321,1.085876e+06
3,173570,6.794024e+05
4,30862,1.251787e+06


# Predicciones para datos no confiables:

In [50]:
# Entrenamos el modelo:
X = train_b.drop('precio', axis=1)
Y = train_b['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [53]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 11,
    'num_leaves': 70,
    #'learning_rate': 0.02,
    'learning_rate': 0.05,
    'verbose': 0, 
    'early_stopping_round': 100}
n_estimators=20000

In [54]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, valid_sets=watchlist, verbose_eval=50)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's l1: 1.00026e+06
[100]	valid_0's l1: 891124
[150]	valid_0's l1: 872183
[200]	valid_0's l1: 863164
[250]	valid_0's l1: 856664
[300]	valid_0's l1: 852965
[350]	valid_0's l1: 850472
[400]	valid_0's l1: 848131
[450]	valid_0's l1: 845008
[500]	valid_0's l1: 843966
[550]	valid_0's l1: 841600
[600]	valid_0's l1: 840469
[650]	valid_0's l1: 838567
[700]	valid_0's l1: 837642
[750]	valid_0's l1: 836588
[800]	valid_0's l1: 836032
[850]	valid_0's l1: 835624
[900]	valid_0's l1: 834890
[950]	valid_0's l1: 834240
[1000]	valid_0's l1: 833377
[1050]	valid_0's l1: 833012
[1100]	valid_0's l1: 832690
[1150]	valid_0's l1: 832619
[1200]	valid_0's l1: 832168
[1250]	valid_0's l1: 831512
[1300]	valid_0's l1: 831049
[1350]	valid_0's l1: 830675
[1400]	valid_0's l1: 830264
[1450]	valid_0's l1: 830283
[1500]	valid_0's l1: 830228
[1550]	valid_0's l1: 830296
[1600]	valid_0's l1: 830338
Early stopping, best iteration is:
[1516]	valid_0's l1: 8

In [55]:
# Realizamos las predicciones para test_b
pred_test_b = reg.predict(test_b.values)

In [56]:
pred_test_b

array([7622446.01228321, 6426612.8913557 , 5565497.28187775, ...,
       8608918.85641762, 8761395.37872929, 9771921.10135002])

In [57]:
test_b['precio'] = pred_test_b

In [58]:
test_b = test_b.reset_index()[['id', 'precio']]

In [59]:
# tenemos test_b
test_b.head()

,id,precio
0,4941,7.622446e+06
1,253578,6.426613e+06
2,208352,5.565497e+06
3,295822,5.276481e+06
4,30763,9.537812e+06


# Join

In [65]:
test_a['id'].nunique()

53287

In [66]:
test_b['id'].nunique()

6713

In [68]:
test = test_a.append(test_b)

In [69]:
test['id'].nunique()

60000

# Submit

In [73]:
ids = test['id'].values
valores = test['precio'].values

In [74]:
escribir_respuesta(ids, valores)